# Mapping world observatories: Plotly Express

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Get-some-location-data." data-toc-modified-id="Get-some-location-data.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get some location data.</a></span></li><li><span><a href="#MapBox-needs-your-personal-access-token." data-toc-modified-id="MapBox-needs-your-personal-access-token.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>MapBox needs your personal access token.</a></span></li></ul></li><li><span><a href="#Whole-earth-projection" data-toc-modified-id="Whole-earth-projection-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Whole earth projection</a></span></li><li><span><a href="#Adding-click-events" data-toc-modified-id="Adding-click-events-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Adding click events</a></span></li><li><span><a href="#Select-a-region" data-toc-modified-id="Select-a-region-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Select a region</a></span></li></ul></div>

## Setup

In [176]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy.io.misc import yaml

import plotly.express as px

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as w

### Get some location data. 

See the Observatory_GeoData notebook for details of generating this.

Pandas dataframe `df` will contain positions in a format suitable for MapBox, dictionary `loc` the astropy coordinates.

In [99]:
df = pd.read_csv('observatories.csv')
df.sample(4)

,name,lat,lon,region
22,La Silla Observatory,-29.256667,-70.730000,SouthAmerica
26,Murchison Widefield Array,-26.703319,116.670815,Australia
13,"Observatorio Astronomico Nacional, San Pedro M...",31.029167,-115.486944,NorthAmerica
7,Palomar,33.356000,-116.863000,NorthAmerica


In [103]:
with open('obs.yaml', 'r') as file:
    loc = yaml.load(file)
display(loc['Kitt Peak'], type(loc['Kitt Peak']))

<EarthLocation (-1994502.60430614, -5037538.54232911, 3358104.99690298) m>

astropy.coordinates.earth.EarthLocation

### MapBox needs your personal access token. 

Get it free from [their website](https://www.mapbox.com/) and store it somewhere easy to find: either your home directory or in its own environment variable. 

In [112]:
home = os.getenv('HOME')
token_path = os.path.join(home, '.mapbox_token')
px.set_mapbox_access_token(open(token_path).read())

## Whole earth projection

Pan and zoom come as standard: use the mouse and look for the menu to appear upper right. Points are colored by region and hover tooltips work, but there are no click events in this map.

In [113]:
fig = px.scatter_geo(df, lat="lat", lon="lon", projection="natural earth",
                      color="region", hover_name="name") 
fig.show()

## Adding click events

So far I only got this to work correctly with a single series, not when the points are split by region.

Plotly Express returns a Figure but click events only work on a FigureWidget, so we need to wrap it. At this point we're outside the supportive environment of Express, and full-fat Plotly gets a bit fiddly.

In [122]:
import plotly.graph_objects as go

fig = px.scatter_geo(df, lat="lat", lon="lon", projection="natural earth",
                      hover_name="name")
fig.update_traces(marker=dict(size=12, color='green'))

f = go.FigureWidget(fig) # to enable click events
scatter = f.data[0]

selection = None

# create our on_click callback function
def update_point(trace, points, selector):
    global selection
    
    # points.point_inds return a list of integer indices to df
    # an iloc gets the df row(s)
    selection = df.iloc[points.point_inds]
    name = selection['name'].values[0]
    region = selection['region'].values[0]
    
    # Set a new title identifying the selection
    f.update_layout(title=f'{name} ({region})')
    
    # color highlighting
    c = ['green',] * len(trace.lat)
    for i in points.point_inds:
        c[i] = 'red'
        with f.batch_update():
            scatter.marker.color = c

# set a callback for the click event        
scatter.on_click(update_point)

# display it
f

FigureWidget({
    'data': [{'geo': 'geo',
              'hoverlabel': {'namelength': 0},
              'hover…

## Select a region

In [167]:
def set_region(selection, show=False):
    if selection is None:
        curr_region = 'NorthAmerica'
    else:
        curr_region = selection['region'].values[0]

    region_df = df[df['region']==curr_region]
    display(region_df.head(3))
    
    if print:
        print(f'{len(region_df.index)} rows total')
    
    return region_df

In [168]:
set_region(selection, True)

,name,lat,lon,region
0,Apache Point,32.780000,-105.820000,NorthAmerica
1,Catalina Observatory,32.416667,-110.731667,NorthAmerica
2,Discovery Channel Telescope,34.744305,-111.422515,NorthAmerica


18 rows total


,name,lat,lon,region
0,Apache Point,32.780000,-105.820000,NorthAmerica
1,Catalina Observatory,32.416667,-110.731667,NorthAmerica
2,Discovery Channel Telescope,34.744305,-111.422515,NorthAmerica
3,Kitt Peak,31.963333,-111.600000,NorthAmerica
4,Lick Observatory,37.343333,-121.636667,NorthAmerica
5,Lowell Observatory,35.096667,-111.535000,NorthAmerica
6,Mt Graham,32.701600,-109.871900,NorthAmerica
7,Palomar,33.356000,-116.863000,NorthAmerica
8,Very Large Array,34.078749,-107.618283,NorthAmerica
9,Whipple Observatory,31.680944,-110.877500,NorthAmerica


In [203]:
def get_ranges():
    height = region_df['lat'].max() - region_df['lat'].min()
    width = region_df['lon'].max() - region_df['lon'].min()
    return height, width

def get_midpoint(region_df):
    midpoint = {}
    midpoint['lat'] = (region_df['lat'].max() + region_df['lat'].min())/2
    midpoint['lon'] = (region_df['lon'].max() + region_df['lon'].min())/2
    return midpoint

In [200]:
regions = list(df.region.unique())

if selection is None:
    curr_region = 'NorthAmerica'
else:
    curr_region = selection['region'].values[0]

regions, curr_region

(['NorthAmerica', 'SouthAmerica', 'Australia', 'EuropeMed', 'Asia', 'Africa'],
 'NorthAmerica')

In [206]:
# height, width = get_ranges()
# midpoint = get_midpoint()

def plot_region(region_df):
    zoom = 3 # still need to find a way to set this intelligently

    fig = px.scatter_mapbox(region_df, lat='lat', lon='lon', hover_name="name",
                      zoom=zoom, center=get_midpoint(region_df), width=800, height=600)
    fig.update_traces(marker=dict(size=10, color='green'))
    fig.show()
    
def update_region(region):
    global curr_region
    curr_region = region
    region_df = df[df['region']==region]
    plot_region(region_df)

In [207]:
style = {'description_width': 'initial'} # to avoid the labels getting truncated
interact(update_region, region = w.RadioButtons(options=regions, value=curr_region,
                                    description='Region:', disabled=False));

interactive(children=(RadioButtons(description='Region:', options=('NorthAmerica', 'SouthAmerica', 'Australia'…